Healthcare Dataset Challenge: The task is to analyze the clinical and financial data of patients hospitalized for a certain condition. It is attached with this email. Some variable names and patient_id's have been anonymized in this dataset. **You are required to join the data given in different tables, and find insights about the drivers of cost of care.** Please see attached documents for datasets.
 
For this challenge, you will be given 1-1.5 weeks, at the end of which we expect you to share your code (github repository) and a document/slides describing your approach and results, insights etc. This position requires strong communication and visualization skills. Your slide design, narrative and content as well your presentation will be judged not only for technical but communication skills as well.



In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
bill_amount = pd.read_csv("/Users/chuadingyuan/Downloads/Healthcare Data Challenge Data/bill_amount.csv")
bill_id = pd.read_csv("/Users/chuadingyuan/Downloads/Healthcare Data Challenge Data/bill_id.csv")
demographics = pd.read_csv("/Users/chuadingyuan/Downloads/Healthcare Data Challenge Data/demographics.csv")
clinical_data = pd.read_csv("/Users/chuadingyuan/Downloads/Healthcare Data Challenge Data/clinical_data.csv")


First start with checking what the data looks like

In [ ]:
print(bill_amount.shape)
print(bill_amount.describe())
bill_amount.head()
# shows the bill id and amount

In [ ]:
print(bill_id.shape)
print(bill_id.describe())
bill_id.head()
# shows bill id,patient id and date of admission

In [ ]:
print(demographics.shape)
print(demographics.describe())
demographics.head()

In [ ]:
print(clinical_data.shape)
print(clinical_data.describe())
clinical_data.head()

Now combine datasets together to analyse

In [ ]:
combined_bill = bill_amount.set_index(['bill_id']).join(bill_id.set_index("bill_id"))
combined_bill = combined_bill.sort_values(by="date_of_admission").sort_values(by='patient_id').reset_index()
print(combined_bill.describe())
combined_bill.head()
# combining the 2 bill dataset for further analysis

In [ ]:
patient_visit_bill = combined_bill.groupby(['patient_id','date_of_admission']).amount.sum().sort_values().reset_index()
print(patient_visit_bill.describe())
patient_visit_bill
# by grouping patient id and date of admission 
# we can find the total bill for each visit

In [ ]:
# function to calculate age based on date of birth
def age(row):
    years = datetime.datetime.today().year - datetime.datetime.strptime(row['date_of_birth'], '%Y-%m-%d').date().year
    return years

# adding age to demographics for further analysis 
demographics['age'] = demographics.apply(lambda row:age(row), axis=1)
print(demographics.shape)
demographics

In [ ]:
# renaming column to standardise
clinical_data = clinical_data.rename(columns = {'id':'patient_id'}).sort_values(by='patient_id')

#sort demographics rows to match clinical_data
demographics = demographics.sort_values(by='patient_id')

# calculating number of times each patient was admitted and adding to dataframe
demographics['admission_count'] = list(clinical_data.groupby(['patient_id']).date_of_admission.count())
demographics[demographics.patient_id == 'f86a66c44517989f2d4c0fffb46eb637']
patient_visit_bill_demo = patient_visit_bill.set_index('patient_id').join(demographics.set_index('patient_id')).reset_index()
patient_visit_bill_demo[patient_visit_bill_demo.patient_id == 'f86a66c44517989f2d4c0fffb46eb637']


In [ ]:
# function to calculate length of stay
def length_of_stay(row):
    admitted = datetime.datetime.strptime(row['date_of_admission'], '%Y-%m-%d').date()
    discharged = datetime.datetime.strptime(row['date_of_discharge'], '%Y-%m-%d').date()
    return (discharged-admitted).days

# adding length of stay of each visit column to dataframe
clinical_data['length_of_stay'] = clinical_data.apply(lambda row:length_of_stay(row), axis=1)
print(clinical_data.describe())
clinical_data

In [ ]:
# function to standardise race string into integers
def race(row):
    if row['race'] == 'Chinese' or row['race'] == 'chinese':
        return 1
    elif row['race'] == 'Indian' or row['race'] == 'India':
        return 2
    elif row['race'] == 'Malay':
        return 3
    elif row['race'] == 'Others':
        return 4

# function to standardise resident status string into integers
def rstatus(row):
    if row['resident_status'] == 'Singaporean' or row['resident_status'] == 'Singapore citizen':
        return 1
    elif row['resident_status'] == 'PR':
        return 2
    elif row['resident_status'] == 'Foreigner':
        return 3

# function to standardise gender string into integers
def gender(row):
    if row['gender'] == 'Male' or row['gender'] == 'm':
        return 1
    elif row['gender'] == 'Female' or row['gender'] =='f':
        return 2

In [ ]:
# to combine all data sets together based on each admission
info_per_visit = clinical_data.set_index(['patient_id','date_of_admission']).join(patient_visit_bill_demo.set_index(['patient_id','date_of_admission'])).reset_index()
info_per_visit['date_of_admission'] = info_per_visit['date_of_admission'].apply(lambda row:datetime.datetime.strptime(row, '%Y-%m-%d').date().month)

# check which columns have na
#print(info_per_visit.isnull().apply(sum))
# get rid of null data
info_per_visit=info_per_visit.fillna(0)

# standardise medical history data into integer
info_per_visit['medical_history_3']=info_per_visit.medical_history_3.replace({"No":0, "Yes":1, "0":0,"1":1})
info_per_visit['medical_history_2']=info_per_visit['medical_history_2'].apply(lambda x:int(x))
info_per_visit['medical_history_5']=info_per_visit['medical_history_5'].apply(lambda x:int(x))

# standardise personal info data into integer 
info_per_visit['race'] = info_per_visit.apply(lambda row:race(row),axis=1)
info_per_visit['resident_status'] = info_per_visit.apply(lambda row:rstatus(row),axis=1)
info_per_visit['gender'] = info_per_visit.apply(lambda row:gender(row),axis=1)


# calculating total number of med history, symptom, preop med of each admission 
info_per_visit['MH_total'] = info_per_visit[['medical_history_1', 'medical_history_2','medical_history_3', 'medical_history_4', 'medical_history_5', 'medical_history_6', 'medical_history_7']].sum(axis=1)
info_per_visit['symptom_total'] = info_per_visit[['symptom_1','symptom_2', 'symptom_3', 'symptom_4', 'symptom_5']].sum(axis=1)
info_per_visit['PM_total'] = info_per_visit[['preop_medication_1', 'preop_medication_2', 'preop_medication_3', 'preop_medication_4', 'preop_medication_5', 'preop_medication_6']].sum(axis=1)

# rearrange the columns for easy reference
# first, get list on columns
print(list(info_per_visit.columns))
# second, rearrange columns
cols = ['patient_id','admission_count','length_of_stay','age','amount', 'MH_total', 'symptom_total', 'PM_total','gender', 'race', 'resident_status', 'symptom_1', 
        'symptom_2', 'symptom_3', 'symptom_4', 'symptom_5','medical_history_1', 'medical_history_2', 
        'medical_history_3', 'medical_history_4', 'medical_history_5', 'medical_history_6', 'medical_history_7', 
        'preop_medication_1', 'preop_medication_2', 'preop_medication_3', 'preop_medication_4', 
        'preop_medication_5', 'preop_medication_6', 'lab_result_1', 'lab_result_2', 'lab_result_3', 'weight', 
        'height',  'date_of_birth','date_of_admission', 'date_of_discharge']

info_per_visit = info_per_visit[cols]
# final combination of all datasets to show total bills, demographics, clinical data 
# and some extra information for each patient id for each visit
info_per_visit

Now we can begin analysing the dataset

In [ ]:
info_per_visit.describe()

In [ ]:
# plot pie charts to get a clearer visualisation
race = info_per_visit.groupby('race').patient_id.count()
print(race)

resident_status = info_per_visit.groupby('resident_status').patient_id.count()
print(resident_status)

#chart for race
pie, ax = plt.subplots(figsize=[10,6])
labels = ['Chinese','Indian','Malay','Others']
plt.pie(x=race, autopct="%.1f%%", explode=[0.05]*4, labels=labels, pctdistance=0.5)
plt.title("Race", fontsize=14);
pie.savefig("Race.png")

#chart for resident_status
pie, ax = plt.subplots(figsize=[10,6])
labels = ['Singaporean','PR','Foreigner']
plt.pie(x=resident_status, autopct="%.1f%%", explode=[0.05]*3, labels=labels, pctdistance=0.5)
plt.title("Resident Status", fontsize=14);
pie.savefig("Resident Status.png")

In [ ]:
# checking for correlation between amount billed and other variables 
print(info_per_visit.corr().amount.sort_values())

# checking correlation for every variable
plt.gcf().set_size_inches((20, 20))
sns.heatmap(info_per_visit.corr())
# medical history 1,6 symptom 1-5

In [ ]:
pd.set_option('max_columns', None)
info_per_visit.sort_values(by="amount",ascending = False).head(100)

In [ ]:
# check whether number of admission contribute to amount paid
admission_vs_amount = info_per_visit.groupby(['patient_id','admission_count']).amount.sum().reset_index()
admission_vs_amount['avg_per_visit']= admission_vs_amount.amount / admission_vs_amount.admission_count

# top 50 avg amount billed per visit
top50 = admission_vs_amount.sort_values(by='avg_per_visit',ascending=False).head(50)
top50

# sanity checks
#info_per_visit[info_per_visit.patient_id == '6f4d80977932f6d83ab5ae491f574c25']
#info_per_visit.sort_values(by='amount',ascending=False)
#admission_vs_amount[admission_vs_amount.patient_id == '6f4d80977932f6d83ab5ae491f574c25']

In [ ]:
print(admission_vs_amount.groupby('admission_count').avg_per_visit.mean())
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

bar, ax = plt.subplots(figsize=[10,6])
labels = [1,2,3,4]
AC = plt.bar(x=labels,height=admission_vs_amount.groupby('admission_count').avg_per_visit.mean().astype(int),
       tick_label=labels)
autolabel(AC)
plt.title("Amount Billed by Admission Count", fontsize=14);
bar.savefig("Amount Billed by Admission Count.png")

In [ ]:
# check if these top 50 billed per visit people have anything in common
top50_clinical_data=info_per_visit.loc[info_per_visit.patient_id.isin(list(top50['patient_id']))]
top50_clinical_data.describe()
print(top50_clinical_data.groupby('race').patient_id.count())
print(top50_clinical_data.groupby('resident_status').patient_id.count())
# preop med 3,5,6 appeard in >76% of patients
# all symptoms appeared in >72% of patients
# 35/50 patients were malay
# 40/50 patients were foreigners

In [ ]:
plt.gcf().set_size_inches((20, 20))
sns.heatmap(top50_clinical_data.corr())
# no variables wtih correlation to amount

In [ ]:
# plotting pie charts to illustrate data

# breakdown of race in top 50
top50_race = top50_clinical_data.groupby('race').patient_id.count()
pie, ax = plt.subplots(figsize=[10,6])
labels = ['Chinese','Indian','Malay','Others']
plt.pie(x=top50_race, autopct="%.1f%%", explode=[0.05]*4, labels=labels, pctdistance=0.5)
plt.title("Top 50 by Race", fontsize=14);
pie.savefig("Top 50 by Race.png")

# breakdown of resident status in top 50
top50_resident = top50_clinical_data.groupby('resident_status').patient_id.count()
pie, ax = plt.subplots(figsize=[10,6])
labels = ['Singaporean','PR','Foreigner']
plt.pie(x=top50_resident, autopct="%.1f%%", explode=[0.05]*3, labels=labels, pctdistance=0.5)
plt.title("Top 50 by Resident_status", fontsize=14);
pie.savefig("Top 50 by Resident_status.png")

# breakdown of average amount billed by race in top 50
bar, ax = plt.subplots(figsize=[10,6])
labels = ['Chinese','Indian','Malay','Others']
plt.bar(x=labels,height=info_per_visit.groupby('race').amount.mean())
plt.title("Amount Billed by Race", fontsize=14);
pie.savefig("Amount Billed by Race.png")

# breakdown of average amount billed by resident status in top 50
bar, ax = plt.subplots(figsize=[10,6])
labels = ['Singaporean','PR','Foreigner']
plt.bar(x=labels, height=info_per_visit.groupby('resident_status').amount.mean())
plt.title("Amount Billed by Resident_status", fontsize=14);
pie.savefig("Amount Billed by Resident_status.png")

print(info_per_visit.groupby('race').amount.mean())
print((29506-(19118+23682+21320)/3)/((19118+23682+21320)/3))
print(info_per_visit.groupby('resident_status').amount.mean())
print((41703-(20211+24370)/2)/((20211+24370)/2))

Splitting Data into groups to compare against amount billed for each visit

length of stay vs amount

personal info vs amount

lab results vs amount

preopmedication vs amount

symptom vs amount

In [ ]:
print(info_per_visit.groupby(['length_of_stay']).amount.mean().sort_values())
info_per_visit.groupby(['length_of_stay']).amount.describe()
plt.plot(info_per_visit.groupby(['length_of_stay']).amount.mean())

In [ ]:
symptom = info_per_visit.loc[:,['amount','symptom_total','symptom_1', 'symptom_2', 'symptom_3','symptom_4', 'symptom_5']]

print(((s1+s2+s3+s4+s5)/5-s0)/s0)
print(symptom.iloc[:,1:].sum())
s0 = symptom.loc[symptom.symptom_total==0].amount.mean()
s1 = symptom.loc[symptom.symptom_1>0].amount.mean()
s2 = symptom.loc[symptom.symptom_2>0].amount.mean()
s3 = symptom.loc[symptom.symptom_3>0].amount.mean()
s4 = symptom.loc[symptom.symptom_4>0].amount.mean()
s5 = symptom.loc[symptom.symptom_5>0].amount.mean()
symptom.sort_values(by='amount',ascending=False).head(100)
print(symptom.groupby('symptom_total').amount.mean())

'''plt.plot(symptom.groupby('symptom_total').amount.mean())
top50_symptom = [s1,s2,s3,s4,s5]
pie, ax = plt.subplots(figsize=[10,6])
labels = ['symptom_1', 'symptom_2', 'symptom_3', 'symptom_4', 'symptom_5']
plt.pie(x=top50_symptom, autopct="%.1f%%", explode=[0.05]*5, labels=labels, pctdistance=0.5)
plt.title("Top 50 by Symptom", fontsize=14);
pie.savefig("Top 50 by Symptom.png")
top50_symptom'''

fig, axs = plt.subplots(1, 2)
plt.gcf().set_size_inches((14,8))

S_T = [s0,s1,s2,s3,s4,s5]
print(S_T)
labels = ['None','symptom_1', 'symptom_2', 'symptom_3', 'symptom_4', 'symptom_5']
axs[0].plot(labels,S_T)
axs[0].set_title("Patients Who Have Symptom No.", fontsize=14);

axs[1].plot(symptom.groupby('symptom_total').amount.mean(),'tab:orange')
axs[1].set_title("No. of Symptoms Per Patient", fontsize=14);


plt.suptitle("Amount Billed vs Symptoms Per Patient Data", fontsize=18);
plt.savefig("Amount Billed vs Symptoms Per Patient Data.png")
# checking symptoms against amount of hospital bill
# if you have symptom 1,2,4 you on average pay 23K, symptom 3 24K, symptom 5 27K
# if patient have all symptoms they pay more, mean =31K
# linear relationship between number of symptoms and amount of bill


In [ ]:
personal_info = info_per_visit.loc[:,['amount','weight', 'height', 'gender', 'race','resident_status','age']]


# plot individual personal info against amount
fig, axs = plt.subplots(2, 3)
plt.gcf().set_size_inches((18,12))
axs[0,0].plot(personal_info.sort_values(by='age').age,personal_info['amount']) 
axs[0,0].set_title('age',fontsize=14)

axs[0,2].plot(['Singaporean','PR','Foreigner'],personal_info.groupby('resident_status').amount.mean(),'tab:orange') 
axs[0,2].set_title('resident_status',fontsize=14)

axs[0,1].plot(['Male','Female'],personal_info.groupby('gender').amount.mean()) 
axs[0,1].set_title('gender',fontsize=14)

axs[1,0].plot(personal_info.groupby('weight').amount.mean())
axs[1,0].set_title('weight',fontsize=14)

axs[1,1].plot(personal_info.groupby('height').amount.mean())
axs[1,1].set_title('height',fontsize=14)

axs[1,2].plot(['Chinese','Indian','Malay','Others'],personal_info.groupby('race').amount.mean(),'tab:orange')
axs[1,2].set_title('race',fontsize=14)

plt.suptitle("Amount Billed vs Personal Info", fontsize=18);
plt.savefig("Amount Billed vs Personal Info.png")

# checking personal info? against amount of hospital bill
# foreigners pay more
# malay pay more

In [ ]:
medical_history = info_per_visit.loc[:,['amount','MH_total','medical_history_1', 'medical_history_2', 'medical_history_3',
                                        'medical_history_4', 'medical_history_5', 'medical_history_6',
                                        'medical_history_7']]
print(((mh1+mh2+mh3+mh4+mh5+mh6+mh7)/7-mh0)/mh0)
print(medical_history.iloc[:,1:].sum())
mh0 = medical_history.loc[medical_history.MH_total == 0].amount.mean()
mh1 = medical_history.loc[medical_history.medical_history_1>0].amount.mean()
mh2 = medical_history.loc[medical_history.medical_history_2>0].amount.mean()
mh3 = medical_history.loc[medical_history.medical_history_3>0].amount.mean()
mh4 = medical_history.loc[medical_history.medical_history_4>0].amount.mean()
mh5 = medical_history.loc[medical_history.medical_history_5>0].amount.mean()
mh6 = medical_history.loc[medical_history.medical_history_6>0].amount.mean()
mh7 = medical_history.loc[medical_history.medical_history_7>0].amount.mean()

print(medical_history.groupby('MH_total').amount.mean())
fig, axs = plt.subplots(1, 2)
plt.gcf().set_size_inches((12,8))

MH_T = [mh0,mh1,mh2,mh3,mh4,mh5,mh6,mh7]
print(MH_T)
labels = [0,1,2,3,4,5,6,7]
axs[0].plot(labels,MH_T)
axs[0].set_title("Patients Who Have Medical History No.", fontsize=14);

axs[1].plot(medical_history.groupby('MH_total').amount.mean(),'tab:orange')
axs[1].set_title("No. of Medical History Per Patient", fontsize=14);


plt.suptitle("Amount Billed vs Medical History Data", fontsize=18);
plt.savefig("Amount Billed vs Medical History Data.png")

# checking medical histories against amount of hospital bill
# 1: 27K, (2,3,4):22K, (5,7):23K, (6):24K

In [ ]:
lab_result = info_per_visit.loc[:,['amount','lab_result_1', 'lab_result_2','lab_result_3']]
print(lab_result.describe())
lab_result
# checking lab_result against amount of hospital bill
# all same bill , no trend either for different levels of lab result

In [ ]:
preop_medication = info_per_visit.loc[:,['amount','PM_total','preop_medication_1', 'preop_medication_2',
                                         'preop_medication_3', 'preop_medication_4', 'preop_medication_5',
                                         'preop_medication_6']]
print(((pm1+pm2+pm3+pm4+pm5+pm6)/6-pm0)/pm0)

# checking preop_medication against amount of hospital bill
# if you have one or more preop med, amount increases linearly
# exlcuding 0 preop medication

print(medical_history.iloc[:,1:].sum())
pm0 = preop_medication.loc[preop_medication.PM_total==0].amount.mean()
pm1 = preop_medication.loc[preop_medication.preop_medication_1>0].amount.mean()
pm2 = preop_medication.loc[preop_medication.preop_medication_2>0].amount.mean()
pm3 = preop_medication.loc[preop_medication.preop_medication_3>0].amount.mean()
pm4 = preop_medication.loc[preop_medication.preop_medication_4>0].amount.mean()
pm5 = preop_medication.loc[preop_medication.preop_medication_5>0].amount.mean()
pm6 = preop_medication.loc[preop_medication.preop_medication_6>0].amount.mean()


print(preop_medication.groupby('PM_total').amount.mean())
fig, axs = plt.subplots(1, 2)
plt.gcf().set_size_inches((10,6))

PM_T = [pm0,pm1,pm2,pm3,pm4,pm5,pm6]
print(PM_T)
labels = [0,1,2,3,4,5,6]
axs[0].plot(PM_T)
axs[0].set_title("Preop Medication No. Used", fontsize=14);

axs[1].plot(preop_medication.groupby('PM_total').amount.mean(),'tab:orange')
axs[1].set_title("No. of Preop Medication Used", fontsize=14);


plt.suptitle("Amount Billed vs Preop Medication Data", fontsize=18);
plt.savefig("Amount Billed vs Preop Medication Data.png")




In [ ]:
lr1 = lab_result.groupby('lab_result_1').amount.mean().sort_values()
lr2 = lab_result.groupby('lab_result_2').amount.mean().sort_values()
lr3 = lab_result.groupby('lab_result_3').amount.mean().sort_values()

PM_T = preop_medication.groupby('PM_total').amount.mean()

fig, axs = plt.subplots(1, 3)
plt.gcf().set_size_inches((12,8))

LR = [lr1,lr2,lr3]
print(LR)
axs[0].plot(lr1)
axs[0].set_title("Amount Billed vs Lab Result 1", fontsize=14);

axs[1].plot(lr2)
axs[1].set_title("Amount Billed vs Lab Result 2", fontsize=14);

axs[2].plot(lr3)
axs[2].set_title("Amount Billed vs Lab Result 3", fontsize=14);


plt.suptitle("Amount Billed vs Lab Results", fontsize=18);
plt.savefig("Amount Billed vs Lab Results.png")

